In [1]:
import os
import torch
import torchvision
from tqdm import tqdm
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pycocotools.coco import COCO

from utils import get_temp_model, tempDataset, tempInferDataset, compute_metrics, InferImgDataset, IoU
from utils import plot_box, clean_box

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:

root = os.getcwd() + '/../data/birds/annotations/all'
list_files = ['test_dataset_220430', 'test_dataset_220502', 'test_dataset_220508', 'test_dataset_220510',
              'test_empty_220430', 'test_empty_220502', 'test_empty_220508', 'test_empty_220510']
num_classes = 2
n_batch = 4

# select device (whether GPU or CPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
model = get_temp_model(num_classes)
model.load_state_dict(torch.load(os.getcwd() + '/../models/bird_detection/outputs/models/output_model_temp_full_v0.pt')['model']) #if only CPU available 
model = model.to(device)
model.eval()
print('... model loaded.')

... model loaded.


In [4]:
torch.cuda.empty_cache()

def get_transform():
    custom_transforms = []
    custom_transforms.append(torchvision.transforms.ToTensor())
    return torchvision.transforms.Compose(custom_transforms)

# collate_fn needs for batch
def collate_fn(batch):
    return tuple(zip(*batch))



# Evaluation

In [5]:
output_res = []

for filename in list_files:
    torch.cuda.empty_cache()

    pathin = root + '/../'+ filename +'.json'

    ds = tempInferDataset(root, pathin, get_transform())
    dataloader = torch.utils.data.DataLoader(ds,
                batch_size = n_batch,
                num_workers = 6,
                shuffle=True,
                collate_fn = collate_fn)

    sum_iou = 0
    metrics = []

    for data in tqdm(dataloader):
        
        imgs, annotations = data
        imgs = list(img.float().to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]

        metrics.append(compute_metrics(model, imgs, annotations))
        
    metrics_arr = np.array([m for m in metrics])
    prec, rec, acc = np.mean(metrics_arr[:,1:], axis = 0)
    miou = np.mean(metrics_arr[:,0][metrics_arr[:,0]!=0])

    output_res.append([filename, len(ds), acc, prec, rec, miou])

arr = np.stack(output_res, axis=0)
new_df = pd.DataFrame(arr, columns = ['dataset', 'num images', 'accuracy', 'precision', 'recall', 'miou'])
new_df.to_csv('metrics_testing_full.csv', index = False)

loading annotations into memory...
Done (t=0.10s)
creating index...
index created!


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:38<00:00,  1.41s/it]


loading annotations into memory...
Done (t=0.12s)
creating index...
index created!


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:35<00:00,  1.21s/it]


loading annotations into memory...
Done (t=0.10s)
creating index...
index created!


...

loading annotations into memory...
Done (t=0.16s)
creating index...
index created!


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 461/461 [10:51<00:00,  1.41s/it]


loading annotations into memory...
Done (t=0.12s)
creating index...
index created!


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 465/465 [08:33<00:00,  1.10s/it]


Bad pipe message: %s [b'\xbf\x19N\x16\x13\xe3d\xb7\x1b\xdc\x88L\x87O\x8f\x98\xc3\x04 O\xd3\x7f\xaa\xbclc\xf3\xfef~\x17\xec\xf8kV\xca\xbdg\xca\x1f\x1cJ\xa64@=\x9f3\xe6\x90\xbe\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06', b'\x07\x08']
Bad pipe message: %s [b'\t\x08\n\x08\x0b\x08\x04']
Bad pipe message: %s [b'D\xc15D\xe7\x95\x83]\xa5Y5\xdf\xe2v\xc6\x85\xb7% \xd6\xe2\xce\xae]0\x9d]\xe6\xa4\x8c\xd8\xb3\x1e\xcb\xdd\xf0\tSt\xbc\x1a<\xb5\x04\x07b\xf2 I\x10}\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x

# Plot outputs

In [6]:
new_df

,dataset,num images,accuracy,precision,recall,miou
0,test_dataset_220430,107,0.8042839105339106,0.8950617283950618,0.8896604938271605,0.7144361352349451
1,test_dataset_220502,116,0.8155747748420162,0.9396551724137931,0.9249999999999999,0.6864216487425692
2,test_dataset_220508,18,0.9583333333333333,1.0,0.9583333333333333,0.7533790384729704
3,test_dataset_220510,3,1.0,1.0,1.0,0.6440846986240811
4,test_empty_220430,1753,0.8445330296127562,0.8445330296127562,0.8445330296127562,nan
5,test_empty_220502,1744,0.9982798165137615,0.9982798165137615,0.9982798165137615,nan
6,test_empty_220508,1842,0.9235357917570499,0.9235357917570499,0.9235357917570499,nan
7,test_empty_220510,1857,0.9967741935483871,0.9967741935483871,0.9967741935483871,nan


In [ ]:
data = next(iter(dataloader))

imgs, annotations = data
imgs = list(img.float().to(device) for img in imgs)
annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
pred = model(imgs)

boxes = []
gt_box = []
scores = []

for j, p in enumerate(pred):
    cond = p['scores'] > 0.6
    boxes.append(p['boxes'][cond])
    scores.append(p['scores'][cond])
    gt_box.append(annotations[j]['boxes'])
    
print('Number of boxes in groundtruth')
print([len(a['boxes']) for a in annotations])

output = [len(b) for b in boxes]
print('Number of boxes predicted')
print(output)

In [ ]:
clean_boxes = clean_box(boxes)
print('Number of boxes after cleaning')
print([len(b) for b in clean_boxes])

In [ ]:
for i in range(len(boxes)):
    np_img = imgs[i][3:6].cpu().numpy().transpose(1,2,0)
    np_box = boxes[i].cpu().detach().numpy()
    np_gt_box = gt_box[i].cpu().detach().numpy()
    plot_box(np_img, np_gt_box)

# Save output boxes

# Notes

boxes : list of all boxes found
res: iou matrix of boxe with box
 --> if iou = 0 means box does not overlapp
 --> if iou > 0.5 means box overlapp : remove the one with the smallest score